In [1]:
import gc
import os
import json
import datasets
from dotenv import load_dotenv
import minds
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import torch
from huggingface_hub import HfApi, HfFolder, login
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings

from honeybee.loaders import (
    PDFreport,
    Scan,
    Slide,
    generate_summary_from_json,
    get_chunk_text,
)
from honeybee.models import REMEDIS, UNI, HuggingFaceEmbedder, TissueDetector

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
load_dotenv()

True

In [2]:
def manifest_to_df(manifest_path, modality):
    with open(manifest_path, "r") as f:
        manifest = json.load(f)

    # Initialize an empty DataFrame for the modality
    modality_df = pd.DataFrame()

    # Process each patient in the manifest
    for patient in manifest:
        patient_id = patient["PatientID"]
        gdc_case_id = patient["gdc_case_id"]

        # Check if the current patient has the requested modality
        if modality in patient:
            # Convert the list of dictionaries into a DataFrame
            df = pd.DataFrame(patient[modality])
            # Add 'PatientID' and 'gdc_case_id' columns
            df["PatientID"] = patient_id
            df["gdc_case_id"] = gdc_case_id

            # Append the new data to the existing DataFrame for this modality
            modality_df = pd.concat([modality_df, df], ignore_index=True)

    # Check if the modality DataFrame is not empty before returning
    if not modality_df.empty:
        return modality_df
    else:
        return None

## Pathology Reports

In [6]:
PROJECTS = [
    "TCGA-ACC",
    "TCGA-COAD",
    # "TCGA-KICH",
    # "TCGA-LIHC",
    # "TCGA-PAAD",
    # "TCGA-SKCM",
    # "TCGA-UCEC",
    # "TCGA-BLCA",
    # "TCGA-DLBC",
    # "TCGA-KIRC",
    # "TCGA-LUAD",
    # "TCGA-PCPG",
    # "TCGA-STAD",
    # "TCGA-UCS",
    # "TCGA-BRCA",
    # "TCGA-ESCA",
    # "TCGA-KIRP",
    # "TCGA-LUSC",
    # "TCGA-PRAD",
    # "TCGA-TGCT",
    # "TCGA-UVM",
    # "TCGA-CESC",
    # "TCGA-GBM",
    # "TCGA-LAML",
    # "TCGA-MESO",
    # "TCGA-READ",
    # "TCGA-THCA",
    # "TCGA-CHOL",
    # "TCGA-HNSC",
    # "TCGA-LGG",
    # "TCGA-OV",
    # "TCGA-SARC",
    # "TCGA-THYM",
]


def setup_writer(parquet_path, schema):
    """Setup or return existing ParquetWriter."""
    if os.path.exists(parquet_path):
        return pq.ParquetWriter(
            parquet_path,
            schema,
            flavor="spark",
            compression="snappy",
            write_mode="append",
        )
    else:
        return pq.ParquetWriter(
            parquet_path, schema, flavor="spark", compression="snappy"
        )


for PROJECT in PROJECTS:
    DATA_DIR = f"/mnt/d/TCGA/raw/{PROJECT}"
    MANIFEST_PATH = DATA_DIR + "/manifest.json"
    MODALITY = "Pathology Report"
    PARQUET = f"/mnt/d/TCGA/parquet/{MODALITY}_gatortron-base.parquet"

    df = manifest_to_df(MANIFEST_PATH, MODALITY)
    embedding_model = HuggingFaceEmbedder(model_name="UFNLP/gatortron-base")
    pdf_report = PDFreport(chunk_size=512, chunk_overlap=10)

    if df is None:
        print(f"No data found for {PROJECT}")
        continue

    writer = None
    schema = None

    for index, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {PROJECT}"):
        try:
            file_path = f"{DATA_DIR}/raw/{row['PatientID']}/{MODALITY}/{row['id']}/{row['file_name']}"
            report_text = pdf_report.load(file_path)

            if report_text:
                embeddings = embedding_model.generate_embeddings(report_text)
                embedding_bytes = embeddings.tobytes()
                df.at[index, "report_text"] = report_text
                df.at[index, "embedding"] = embedding_bytes
                df.at[index, "embedding_shape"] = embeddings.shape
            else:
                raise ValueError("No report text loaded.")

        except Exception as e:
            print(f"Error processing {row['PatientID']}: {e}")

        table = pa.Table.from_pandas(df.iloc[[index]])
        if writer is None or not table.schema.equals(schema):
            if writer is not None:
                writer.close()
            schema = table.schema
            writer = setup_writer(PARQUET, schema)

        writer.write_table(table)

    if writer is not None:
        writer.close()

    gc.collect()
    torch.cuda.empty_cache()

Processing TCGA-ACC:   0%|          | 0/92 [00:00<?, ?it/s]

Error processing TCGA-P6-A5OG: Must have equal len keys and value when setting with an iterable


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:93                                                                                   │
│                                                                                                  │
│    90 │   │   │   if writer is not None:                                                         │
│    91 │   │   │   │   writer.close()                                                             │
│    92 │   │   │   schema = table.schema                                                          │
│ ❱  93 │   │   │   writer = setup_writer(PARQUET, schema)                                         │
│    94 │   │                                                                                      │
│    95 │   │   writer.write_table(table)                                                          │
│    96                                                                                            │
│                                                                                                  │
│ in setup_writer:49                                                                               │
│                                                                                                  │
│    46 │   │   │   write_mode="append",                                                           │
│    47 │   │   )                                                                                  │
│    48 │   else:                                                                                  │
│ ❱  49 │   │   return pq.ParquetWriter(                                                           │
│    50 │   │   │   parquet_path, schema, flavor="spark", compression="snappy"                     │
│    51 │   │   )                                                                                  │
│    52                                                                                            │
│                                                                                                  │
│ /home/aakash/.local/lib/python3.10/site-packages/pyarrow/parquet/core.py:1004 in __init__        │
│                                                                                                  │
│   1001 │   │   │   │   # ARROW-10480: do not auto-detect compression.  While                     │
│   1002 │   │   │   │   # a filename like foo.parquet.gz is nonconforming, it                     │
│   1003 │   │   │   │   # shouldn't implicitly apply compression.                                 │
│ ❱ 1004 │   │   │   │   sink = self.file_handle = filesystem.open_output_stream(                  │
│   1005 │   │   │   │   │   path, compression=None)                                               │
│   1006 │   │   else:                                                                             │
│   1007 │   │   │   sink = where                                                                  │
│                                                                                                  │
│ in pyarrow._fs.FileSystem.open_output_stream:878                                                 │
│                                                                                                  │
│ in pyarrow.lib.pyarrow_internal_check_status:154                                                 │
│                                                                                                  │
│ in pyarrow.lib.check_status:91                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] Failed to open local file '/mnt/d/TCGA/parquet/Pathology 
Report_gatortron-base.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
DATA_DIR = "/mnt/d/TCGA-LUAD"
MANIFEST_PATH = "/mnt/d/TCGA-LUAD/manifest.json"
MODALITY = "Pathology Report"
PARQUET = f"/mnt/d/TCGA-LUAD/parquet/{MODALITY}.parquet"

df = manifest_to_df(MANIFEST_PATH, MODALITY)
embedding_model = HuggingFaceEmbedder(model_name="UFNLP/gatortron-base")
pdf_report = PDFreport(chunk_size=512, chunk_overlap=10)

report_texts = []
df["report_text"] = None
df["embedding"] = None
df["embedding_shape"] = None

writer = None
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    try:
        file_path = f"{DATA_DIR}/raw/{row['PatientID']}/{MODALITY}/{row['id']}/{row['file_name']}"
        report_text = pdf_report.load(file_path)
        report_texts.append(report_text)

        if len(report_text) > 0:
            embeddings = embedding_model.generate_embeddings(report_text)
            df.at[index, "embedding_shape"] = embeddings.shape
            embeddings = embeddings.reshape(-1)
            embeddings = embeddings.tobytes()
            df.at[index, "embedding"] = embeddings
        else:
            df.at[index, "embedding"] = None
        df.at[index, "report_text"] = report_text

    except Exception as e:
        print(f"Error: {e}")
        report_texts.append(None)
        df.at[index, "embedding"] = None

    if writer is None:
        schema = pa.Table.from_pandas(df.iloc[[index]]).schema
        writer = pq.ParquetWriter(PARQUET, schema)

    table = pa.Table.from_pandas(df.iloc[[index]], schema=schema)
    try:
        writer.write_table(table)
    except ValueError as e:
        print(f"Schema mismatch error: {e}")
        schema = table.schema
        writer = pq.ParquetWriter(PARQUET, schema)
        writer.write_table(table)

if writer is not None:
    writer.close()

gc.collect()
torch.cuda.empty_cache()

## Slide Image

In [ ]:
DATA_DIR = "/mnt/d/TCGA-LUAD"
MANIFEST_PATH = "/mnt/d/TCGA-LUAD/manifest.json"
MODALITY = "Slide Image"
PARQUET = f"/mnt/d/TCGA-LUAD/parquet/{MODALITY}.parquet"
HE_DETECTOR_PATH = "/mnt/f/Projects/Multimodal-Transformer/models/deep-tissue-detector_densenet_state-dict.pt"
EMBEDDING_MODEL_PATH = (
    "/mnt/d/ckpts/vit_large_patch16_224.dinov2.uni_mass100k/pytorch_model.bin"
)

df = manifest_to_df(MANIFEST_PATH, MODALITY)
tissue_detector = TissueDetector(model_path=HE_DETECTOR_PATH)
embedding_model_path = EMBEDDING_MODEL_PATH
uni = UNI()

df["embedding"] = None
df["embedding_shape"] = None
writer = None
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    try:
        slide_image_path = f"{DATA_DIR}/raw/{row['PatientID']}/{MODALITY}/{row['id']}/{row['file_name']}"
        slide = Slide(
            slide_image_path,
            tileSize=512,
            max_patches=100,
            visualize=False,
            tissue_detector=tissue_detector,
        )
        patches = slide.load_patches_concurrently(target_patch_size=224)

        if patches.shape[0] == 0:
            slide = Slide(
                slide_image_path,
                tileSize=512,
                max_patches=1000,
                visualize=True,
                tissue_detector=tissue_detector,
            )
            patches = slide.load_patches_concurrently(target_patch_size=224)

            if patches.shape[0] == 0:
                with open("errors.txt", "a") as f:
                    f.write(f"{slide_image_path} | No patches extracted.\n")
                raise ValueError("No patches extracted.")

        embedding = uni.load_model_and_predict(embedding_model_path, patches)
        df.at[index, "embedding_shape"] = embedding.shape
        embedding = embedding.reshape(-1)
        embedding = np.array(embedding, dtype=np.float32)
        embedding = embedding.tobytes()
        df.at[index, "embedding"] = embedding
    except Exception as e:
        with open("errors.txt", "a") as f:
            f.write(f"{slide_image_path} | {e}\n")
        df.at[index, "embedding"] = None
        continue

    if writer is None:
        table = pa.Table.from_pandas(df.iloc[[index]])
        writer = pq.ParquetWriter(PARQUET, table.schema)
    else:
        table = pa.Table.from_pandas(df.iloc[[index]])
        writer.write_table(table)

    del slide, patches, embedding, table
    gc.collect()
    torch.cuda.empty_cache()

if writer is not None:
    writer.close()

## CT

In [ ]:
DATA_DIR = "/mnt/d/TCGA/raw/TCGA-LUAD"
MANIFEST_PATH = "/mnt/d/TCGA/raw/TCGA-LUAD/manifest.json"
MODALITY = "CT"
PARQUET = f"/mnt/d/TCGA-LUAD/parquet/{MODALITY}.parquet"

df = manifest_to_df(MANIFEST_PATH, MODALITY)

# --- CONFIGURATION ---
embedding_model_path = "/mnt/d/Models/REMEDIS/onnx/cxr-50x1-remedis-m.onnx"

# Define a consistent schema
schema = pa.schema(
    [
        ("StudyInstanceUID", pa.string()),
        ("SeriesInstanceUID", pa.string()),
        ("SeriesDate", pa.string()),
        ("BodyPartExamined", pa.string()),
        ("SeriesNumber", pa.string()),
        ("Collection", pa.string()),
        ("Manufacturer", pa.string()),
        ("ManufacturerModelName", pa.string()),
        ("SoftwareVersions", pa.string()),
        ("Visibility", pa.string()),
        ("ImageCount", pa.int64()),
        ("PatientID", pa.string()),
        ("gdc_case_id", pa.string()),
        ("ProtocolName", pa.string()),
        ("SeriesDescription", pa.string()),
        ("embedding", pa.binary()),
        ("embedding_shape", pa.list_(pa.int64())),
        ("__index_level_0__", pa.int64()),
    ]
)

df["embedding"] = None
df["embedding_shape"] = None
writer = None
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    try:
        file_path = f"{DATA_DIR}/raw/{row['PatientID']}/{MODALITY}/{row['SeriesInstanceUID']}/{row['SeriesInstanceUID']}"
        scanner = Scan(file_path, modality="CT")
        patches = scanner.load_patches(target_patch_size=448)
        embedding = REMEDIS.load_model_and_predict(embedding_model_path, patches)
        df.at[index, "embedding_shape"] = embedding.shape
        embedding = embedding.reshape(-1)
        embedding = embedding.tobytes()
        df.at[index, "embedding"] = embedding
    except Exception as e:
        print(f"\033[91mError: {e}\033[0m")
        df.at[index, "embedding"] = None
        scanner = None
        patches = None
        embedding = None
        table = None

    if writer is None:
        table = pa.Table.from_pandas(df.iloc[[index]])
        writer = pq.ParquetWriter(PARQUET, schema)
    else:
        table = pa.Table.from_pandas(df.iloc[[index]], schema=schema)
        writer.write_table(table)

    del scanner, patches, embedding, table
    gc.collect()
    torch.cuda.empty_cache()

if writer is not None:
    writer.close()

## Clinical Data

In [ ]:
def process_group(group):
    common_fields = {}
    nested_objects = []
    for col in group.columns:
        unique_values = group[col].dropna().unique()
        if len(unique_values) == 1:
            # If only one unique value exists, it's a common field
            common_fields[col] = unique_values[0]

    # Create nested objects for fields that are not common
    for idx, row in group.iterrows():
        nested_object = {
            col: row[col]
            for col in group.columns
            if col not in common_fields and pd.notna(row[col])
        }
        if nested_object:  # Only add if the nested object is not empty
            nested_objects.append(nested_object)

    return common_fields, nested_objects


PROJECTS = [
    "TCGA-ACC",
    "TCGA-COAD",
    "TCGA-KICH",
    "TCGA-LIHC",
    "TCGA-PAAD",
    "TCGA-SKCM",
    "TCGA-UCEC",
    "TCGA-BLCA",
    "TCGA-DLBC",
    "TCGA-KIRC",
    "TCGA-LUAD",
    "TCGA-PCPG",
    "TCGA-STAD",
    "TCGA-UCS",
    "TCGA-BRCA",
    "TCGA-ESCA",
    "TCGA-KIRP",
    "TCGA-LUSC",
    "TCGA-PRAD",
    "TCGA-TGCT",
    "TCGA-UVM",
    "TCGA-CESC",
    "TCGA-GBM",
    "TCGA-LAML",
    "TCGA-MESO",
    "TCGA-READ",
    "TCGA-THCA",
    "TCGA-CHOL",
    "TCGA-HNSC",
    "TCGA-LGG",
    "TCGA-OV",
    "TCGA-SARC",
    "TCGA-THYM",
]

embedding_model = HuggingFaceEmbedder(model_name="UFNLP/gatortron-medium")

for PROJECT in PROJECTS:
    print(f"Processing {PROJECT}")
    DATA_DIR = f"/mnt/d/TCGA/raw/{PROJECT}"
    MANIFEST_PATH = DATA_DIR + "/manifest.json"
    MODALITY = "Clinical Data"
    PARQUET = f"/mnt/d/TCGA/parquet/{PROJECT}/{MODALITY}.parquet"

    tables = minds.get_tables()
    json_objects = {}
    for table in tqdm(tables, desc="Getting data from tables"):
        query = f"SELECT * FROM minds.{table} WHERE project_id='{PROJECT}'"
        df = minds.query(query)
        for case_id, group in tqdm(df.groupby("case_submitter_id"), leave=False):
            if case_id not in json_objects:
                json_objects[case_id] = {}
            common_fields, nested_objects = process_group(group)
            json_objects[case_id].update(common_fields)
            json_objects[case_id][table] = nested_objects

    df = []
    for case_id, patient_data in tqdm(json_objects.items()):
        summary = generate_summary_from_json(patient_data)
        if len(summary) > 0:
            summary_chunks = get_chunk_text(summary)
            chunk_embeddings = []
            for chunk in summary_chunks:
                chunk_embedding = embedding_model.generate_embeddings([chunk])
                chunk_embeddings.append(chunk_embedding)
            clinical_embedding = np.array(chunk_embeddings)
        else:
            clinical_embedding = None
        patient_data["text"] = summary
        patient_data["embedding_shape"] = clinical_embedding.shape
        clinical_embedding = clinical_embedding.reshape(-1)
        clinical_embedding = np.array(clinical_embedding, dtype=np.float32)
        clinical_embedding = clinical_embedding.tobytes()
        patient_data["embedding"] = clinical_embedding
        # Create a new dictionary for DataFrame conversion, excluding lists
        patient_data_for_df = {
            key: value
            for key, value in patient_data.items()
            if not isinstance(value, list)
        }
        df.append(patient_data_for_df)

    clinical_df = pd.DataFrame(df)
    clinical_df.to_parquet(PARQUET, index=False)

## Uploading datasets to huggingface

### Upload the Local HF datasets to HuggingFace datasets hub

In [ ]:
# api = HfApi()
# api.upload_folder(
#     folder_path="/mnt/d/TCGA-LUAD/parquet/",
#     repo_id="aakashtripathi/TCGA-LUAD",
#     repo_type="dataset",
#     multi_commits=True,
#     multi_commits_verbose=True,
# )

## Loading Dataset from HuggingFace into PyTorch Dataloader